# Imports & Load Data
作業に必要なライブラリをインポートして、 以下のデータを読み込みます。

* stock_price : 株価情報
* stock_list : 銘柄情報
* stock_fin : 財務諸表
* stock_labels : 目的変数

In [3]:
!pip install catboost==0.24.4 

  Using cached https://files.pythonhosted.org/packages/96/3b/bb419654adcf7efff42ed8a3f84e50c8f236424b7ed1cc8ccd290852e003/catboost-0.24.4-cp37-none-manylinux1_x86_64.whl
    100% |████████████████████████████████| 13.2MB 4.4MB/s eta 0:00:011 day, 23:59:59
  Stored in directory: /root/.cache/pip/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying


In [4]:
import os
import pickle
import sys
import warnings
from glob import glob

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import shap
import  xgboost as xgb
from scipy.stats import spearmanr
from sklearn.ensemble import (
    ExtraTreesRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor,
)
from sklearn.metrics import accuracy_score, mean_squared_error
from tqdm.auto import tqdm
from catboost import CatBoostRegressor


# 表示用の設定を変更します
%matplotlib inline
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
pd.options.display.width = 120

In [5]:
# python 3.7.3であることを確認します
import sys
print(sys.version)

3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]


In [6]:
# データセット保存先ディレクトリ（""の中身はご自身の環境に合わせて定義してください。）
dataset_dir="/path/to"

In [7]:
# 読み込むファイルを定義します。
inputs = {
    "stock_list": f"{dataset_dir}/stock_list.csv.gz",
    "stock_price": f"{dataset_dir}/stock_price.csv.gz",
    "stock_fin": f"{dataset_dir}/stock_fin.csv.gz",
    # 本チュートリアルでは使用しないため、コメントアウトしています。
    # "stock_fin_price": f"{dataset_dir}/stock_fin_price.csv.gz",
    "stock_labels": f"{dataset_dir}/stock_labels.csv.gz",
}

# ファイルを読み込みます
dfs = {}
for k, v in inputs.items():
    print(k)
    dfs[k] = pd.read_csv(v)

stock_list
stock_price
stock_fin
stock_labels


# 特徴量の生成

In [13]:
FEATURES = ['MA_gap_2month',
            'MA_gap_3month',
            'volatility_2month',
            'volatility_3month',
            'Result_Dividend FiscalYear',
            'return_3month',
            'Forecast_Dividend FiscalYear',
            'volatility_1month',
            'Forecast_FinancialStatement FiscalYear',
            'MA_gap_1month',
            'pbr',
            'Result_FinancialStatement FiscalYear',
            'return_1month',
            'ema_12',
            'Result_FinancialStatement TotalAssets',
            'signal',
            'Previous_FinancialStatement NetIncome',
            'per',
            'Result_FinancialStatement CashFlowsFromOperatingActivities',
            'Result_FinancialStatement CashFlowsFromInvestingActivities',
            'ema_10']

In [14]:
def get_feature_columns(dfs, train_X, column_group="fundamental+technical"):
    # 特徴量グループを定義
    # ファンダメンタル
    fundamental_cols = dfs["stock_fin"].select_dtypes("float64").columns
    fundamental_cols = fundamental_cols[
        fundamental_cols != "Result_Dividend DividendPayableDate"
        ]
    fundamental_cols = fundamental_cols[fundamental_cols != "Local Code"]
    # 価格変化率
    returns_cols = [x for x in train_X.columns if "return" in x]
    # テクニカル
    technical_cols = [
        x for x in train_X.columns if
        (x not in fundamental_cols) and (x != "code")
    ]
    columns = {
        "fundamental_only": fundamental_cols,
        "return_only": returns_cols,
        "technical_only": technical_cols,
        "fundamental+technical": list(fundamental_cols) + list(
            technical_cols),
    }
    return columns[column_group]

In [15]:
model_path = os.path.join(os.path.dirname("__file__"), "../model")
test_X_path = os.path.join(os.path.dirname("__file__"), "../model/proceed_datas/test_X")
test_y_path = os.path.join(os.path.dirname("__file__"), "../model/proceed_datas/test_y")
val_X_path = os.path.join(os.path.dirname("__file__"), "../model/proceed_datas/val_X")
val_y_path = os.path.join(os.path.dirname("__file__"), "../model/proceed_datas/val_y")
train_X_path = os.path.join(os.path.dirname("__file__"), "../model/proceed_datas/train_X")
train_y_path = os.path.join(os.path.dirname("__file__"), "../model/proceed_datas/train_y")

In [16]:
# 対象の目的変数を定義
labels = {
#    "label_high_5",
#    "label_high_10",
    "label_high_20",
#    "label_low_5",
#    "label_low_10",
    "label_low_20",
}

In [17]:
# 特徴量追加済みデータ
proceed_datas = {
    "train_X",
    "train_y",
    "val_X",
    "val_y",
    "test_X",
    "test_y"
}

In [29]:
# ライブラリインポート
from sklearn.model_selection import GridSearchCV

'''
reg_cv = GridSearchCV(xgb, {
    "eta": [0.01, 0.05, 0.1], 
    "gamma": [0.1,0.2,0.3,0.4,0.5],
    "n_estimators": [50, 100, 200], 
    "max_depth": [5, 7, 9,10,20,30],
    "subsample":[0.6,0.8,1],
    "colsample_bytree": [0.5,0.7,0.9],
}, verbose=1)
'''
reg_cv = GridSearchCV(CatBoostRegressor(), {'iterations': [700], 'depth': [5], 'learning_rate': [0.01], 
        'random_seed': [0]
    }, verbose=1)


In [30]:
for label in tqdm(labels):
    data_X = os.path.join(train_X_path, f"train_X_{label}.pkl")
    with open(data_X , "rb") as f:
        train_X = pickle.load(f)
    data_y = os.path.join(train_y_path, f"train_y_{label}.pkl")
    with open(data_y , "rb") as f:
        train_y = pickle.load(f)
        
    feature_columns = get_feature_columns(dfs, train_X, column_group='fundamental+technical')
    # 訓練実施
    reg_cv.fit(train_X[feature_columns].values, train_y.values)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0:	learn: 0.1608832	total: 7.35ms	remaining: 5.13s
1:	learn: 0.1607743	total: 13.7ms	remaining: 4.76s
2:	learn: 0.1606718	total: 19.8ms	remaining: 4.61s
3:	learn: 0.1605685	total: 25.2ms	remaining: 4.38s
4:	learn: 0.1604579	total: 30.8ms	remaining: 4.29s
5:	learn: 0.1603499	total: 38.8ms	remaining: 4.49s
6:	learn: 0.1602440	total: 46.6ms	remaining: 4.62s
7:	learn: 0.1601390	total: 52.4ms	remaining: 4.53s
8:	learn: 0.1600427	total: 57.2ms	remaining: 4.39s
9:	learn: 0.1599455	total: 62.9ms	remaining: 4.34s
10:	learn: 0.1598446	total: 68.2ms	remaining: 4.27s
11:	learn: 0.1597439	total: 76.7ms	remaining: 4.4s
12:	learn: 0.1596447	total: 83.7ms	remaining: 4.42s
13:	learn: 0.1595664	total: 89.7ms	remaining: 4.39s
14:	learn: 0.1594710	total: 95.3ms	remaining: 4.35s
15:	learn: 0.1593839	total: 101ms	remaining: 4.31s
16:	learn: 0.1593075	total: 106ms	remaining: 4.26s
17:	learn: 0.1592174	total: 115ms	remaining: 4.34s
18:	learn: 0.1591285	total: 122ms	remaining: 4.39s
19:	learn: 0.1590485	total:

185:	learn: 0.1521638	total: 1.05s	remaining: 2.9s
186:	learn: 0.1521328	total: 1.06s	remaining: 2.9s
187:	learn: 0.1521090	total: 1.06s	remaining: 2.89s
188:	learn: 0.1520951	total: 1.07s	remaining: 2.89s
189:	learn: 0.1520787	total: 1.07s	remaining: 2.88s
190:	learn: 0.1520599	total: 1.08s	remaining: 2.87s
191:	learn: 0.1520356	total: 1.08s	remaining: 2.86s
192:	learn: 0.1520105	total: 1.09s	remaining: 2.85s
193:	learn: 0.1519852	total: 1.09s	remaining: 2.85s
194:	learn: 0.1519687	total: 1.1s	remaining: 2.84s
195:	learn: 0.1519502	total: 1.1s	remaining: 2.83s
196:	learn: 0.1519300	total: 1.1s	remaining: 2.82s
197:	learn: 0.1519131	total: 1.11s	remaining: 2.81s
198:	learn: 0.1518939	total: 1.11s	remaining: 2.8s
199:	learn: 0.1518698	total: 1.11s	remaining: 2.79s
200:	learn: 0.1518491	total: 1.12s	remaining: 2.78s
201:	learn: 0.1518329	total: 1.12s	remaining: 2.77s
202:	learn: 0.1518136	total: 1.13s	remaining: 2.77s
203:	learn: 0.1517956	total: 1.13s	remaining: 2.76s
204:	learn: 0.1517

348:	learn: 0.1493195	total: 1.93s	remaining: 1.95s
349:	learn: 0.1493062	total: 1.94s	remaining: 1.94s
350:	learn: 0.1492938	total: 1.95s	remaining: 1.94s
351:	learn: 0.1492828	total: 1.95s	remaining: 1.93s
352:	learn: 0.1492696	total: 1.96s	remaining: 1.92s
353:	learn: 0.1492571	total: 1.96s	remaining: 1.92s
354:	learn: 0.1492492	total: 1.97s	remaining: 1.91s
355:	learn: 0.1492355	total: 1.97s	remaining: 1.9s
356:	learn: 0.1492256	total: 1.98s	remaining: 1.9s
357:	learn: 0.1492091	total: 1.98s	remaining: 1.89s
358:	learn: 0.1491961	total: 1.98s	remaining: 1.88s
359:	learn: 0.1491763	total: 1.99s	remaining: 1.88s
360:	learn: 0.1491593	total: 1.99s	remaining: 1.87s
361:	learn: 0.1491440	total: 2s	remaining: 1.86s
362:	learn: 0.1491369	total: 2s	remaining: 1.86s
363:	learn: 0.1491256	total: 2s	remaining: 1.85s
364:	learn: 0.1491120	total: 2.01s	remaining: 1.84s
365:	learn: 0.1490950	total: 2.01s	remaining: 1.84s
366:	learn: 0.1490874	total: 2.02s	remaining: 1.83s
367:	learn: 0.1490784	t

...

0:	learn: 0.0761016	total: 10.8ms	remaining: 7.57s
1:	learn: 0.0759898	total: 19.4ms	remaining: 6.76s
2:	learn: 0.0758885	total: 27.8ms	remaining: 6.47s
3:	learn: 0.0757790	total: 35.8ms	remaining: 6.23s
4:	learn: 0.0756746	total: 46.1ms	remaining: 6.41s
5:	learn: 0.0755636	total: 57.2ms	remaining: 6.61s
6:	learn: 0.0754606	total: 66ms	remaining: 6.53s
7:	learn: 0.0753565	total: 71.3ms	remaining: 6.17s
8:	learn: 0.0752568	total: 77.3ms	remaining: 5.93s
9:	learn: 0.0751602	total: 84.8ms	remaining: 5.85s
10:	learn: 0.0750599	total: 93.4ms	remaining: 5.85s
11:	learn: 0.0749608	total: 102ms	remaining: 5.87s
12:	learn: 0.0748676	total: 110ms	remaining: 5.83s
13:	learn: 0.0747789	total: 117ms	remaining: 5.75s
14:	learn: 0.0746858	total: 124ms	remaining: 5.66s
15:	learn: 0.0745926	total: 133ms	remaining: 5.7s
16:	learn: 0.0745085	total: 142ms	remaining: 5.71s
17:	learn: 0.0744171	total: 149ms	remaining: 5.64s
18:	learn: 0.0743302	total: 156ms	remaining: 5.58s
19:	learn: 0.0742449	total: 161ms

163:	learn: 0.0688412	total: 1.23s	remaining: 4.02s
164:	learn: 0.0688275	total: 1.24s	remaining: 4.02s
165:	learn: 0.0688134	total: 1.25s	remaining: 4.02s
166:	learn: 0.0688005	total: 1.26s	remaining: 4.01s
167:	learn: 0.0687873	total: 1.27s	remaining: 4.01s
168:	learn: 0.0687743	total: 1.27s	remaining: 4.01s
169:	learn: 0.0687617	total: 1.28s	remaining: 4s
170:	learn: 0.0687475	total: 1.29s	remaining: 4s
171:	learn: 0.0687320	total: 1.3s	remaining: 4s
172:	learn: 0.0687199	total: 1.31s	remaining: 3.99s
173:	learn: 0.0687067	total: 1.32s	remaining: 3.98s
174:	learn: 0.0686935	total: 1.33s	remaining: 3.98s
175:	learn: 0.0686787	total: 1.34s	remaining: 3.98s
176:	learn: 0.0686654	total: 1.35s	remaining: 3.98s
177:	learn: 0.0686531	total: 1.35s	remaining: 3.97s
178:	learn: 0.0686432	total: 1.36s	remaining: 3.97s
179:	learn: 0.0686274	total: 1.37s	remaining: 3.97s
180:	learn: 0.0686148	total: 1.38s	remaining: 3.96s
181:	learn: 0.0686018	total: 1.39s	remaining: 3.96s
182:	learn: 0.0685895	

341:	learn: 0.0673065	total: 2.63s	remaining: 2.75s
342:	learn: 0.0672992	total: 2.64s	remaining: 2.75s
343:	learn: 0.0672948	total: 2.65s	remaining: 2.74s
344:	learn: 0.0672907	total: 2.65s	remaining: 2.73s
345:	learn: 0.0672863	total: 2.66s	remaining: 2.72s
346:	learn: 0.0672815	total: 2.67s	remaining: 2.71s
347:	learn: 0.0672776	total: 2.67s	remaining: 2.71s
348:	learn: 0.0672731	total: 2.68s	remaining: 2.7s
349:	learn: 0.0672686	total: 2.69s	remaining: 2.69s
350:	learn: 0.0672648	total: 2.69s	remaining: 2.68s
351:	learn: 0.0672591	total: 2.7s	remaining: 2.67s
352:	learn: 0.0672530	total: 2.71s	remaining: 2.66s
353:	learn: 0.0672477	total: 2.71s	remaining: 2.65s
354:	learn: 0.0672427	total: 2.72s	remaining: 2.64s
355:	learn: 0.0672388	total: 2.73s	remaining: 2.63s
356:	learn: 0.0672340	total: 2.73s	remaining: 2.63s
357:	learn: 0.0672292	total: 2.74s	remaining: 2.62s
358:	learn: 0.0672251	total: 2.75s	remaining: 2.61s
359:	learn: 0.0672159	total: 2.75s	remaining: 2.6s
360:	learn: 0.0

524:	learn: 0.0664847	total: 3.82s	remaining: 1.27s
525:	learn: 0.0664825	total: 3.83s	remaining: 1.27s
526:	learn: 0.0664785	total: 3.84s	remaining: 1.26s
527:	learn: 0.0664727	total: 3.84s	remaining: 1.25s
528:	learn: 0.0664697	total: 3.85s	remaining: 1.24s
529:	learn: 0.0664646	total: 3.86s	remaining: 1.24s
530:	learn: 0.0664625	total: 3.86s	remaining: 1.23s
531:	learn: 0.0664577	total: 3.87s	remaining: 1.22s
532:	learn: 0.0664535	total: 3.87s	remaining: 1.21s
533:	learn: 0.0664502	total: 3.88s	remaining: 1.21s
534:	learn: 0.0664466	total: 3.89s	remaining: 1.2s
535:	learn: 0.0664432	total: 3.89s	remaining: 1.19s
536:	learn: 0.0664394	total: 3.9s	remaining: 1.18s
537:	learn: 0.0664343	total: 3.9s	remaining: 1.18s
538:	learn: 0.0664303	total: 3.91s	remaining: 1.17s
539:	learn: 0.0664269	total: 3.92s	remaining: 1.16s
540:	learn: 0.0664221	total: 3.93s	remaining: 1.15s
541:	learn: 0.0664183	total: 3.93s	remaining: 1.15s
542:	learn: 0.0664138	total: 3.94s	remaining: 1.14s
543:	learn: 0.0

696:	learn: 0.0659145	total: 4.98s	remaining: 21.4ms
697:	learn: 0.0659121	total: 4.99s	remaining: 14.3ms
698:	learn: 0.0659094	total: 5s	remaining: 7.14ms
699:	learn: 0.0659071	total: 5s	remaining: 0us



In [31]:
# 結果を表示
print(reg_cv.best_params_)
print(reg_cv.best_score_)

{'depth': 5, 'iterations': 700, 'learning_rate': 0.01, 'random_seed': 0}
0.2134283144640482


In [32]:
best_model = reg_cv.best_estimator_

In [33]:
SELECT_FIN_DATA_COLUMNS = ['Result_FinancialStatement FiscalYear', 'Result_FinancialStatement NetSales',
       'Result_FinancialStatement OperatingIncome', 'Result_FinancialStatement OrdinaryIncome',
       'Result_FinancialStatement NetIncome', 'Result_FinancialStatement TotalAssets',
       'Result_FinancialStatement NetAssets', 'Result_FinancialStatement CashFlowsFromOperatingActivities',
       'Result_FinancialStatement CashFlowsFromFinancingActivities',
       'Result_FinancialStatement CashFlowsFromInvestingActivities', 'Forecast_FinancialStatement FiscalYear',
       'Forecast_FinancialStatement NetSales', 'Forecast_FinancialStatement OperatingIncome',
       'Forecast_FinancialStatement OrdinaryIncome', 'Forecast_FinancialStatement NetIncome',
       'Result_Dividend FiscalYear', 'Result_Dividend QuarterlyDividendPerShare',
       'Result_Dividend AnnualDividendPerShare', 'Forecast_Dividend FiscalYear',
       'Forecast_Dividend QuarterlyDividendPerShare', 'Forecast_Dividend AnnualDividendPerShare',
       'IssuedShareEquityQuote IssuedShare','Section/Products', '33 Sector(Code)', '17 Sector(Code)']

In [34]:
# 学習用データセット定義
# ファンダメンタル情報
#fundamental_cols = dfs["stock_fin"].select_dtypes("float64").columns
fundamental_cols = pd.Index(SELECT_FIN_DATA_COLUMNS)
fundamental_cols = fundamental_cols[fundamental_cols != "Result_Dividend DividendPayableDate"]
fundamental_cols = fundamental_cols[fundamental_cols != "Local Code"]
# 価格変化率
returns_cols = [x for x in train_X.columns if "return" in x]
# テクニカル
technical_cols = [x for x in train_X.columns if (x not in fundamental_cols) and (x != "code")]

In [35]:
columns = {
    "fundamental_only": fundamental_cols,
    "return_only": returns_cols,
    "technical_only": technical_cols,
    "fundamental+technical": list(fundamental_cols) + list(technical_cols),
     "selected_columns":FEATURES,
}

In [36]:
# 結果保存用
all_results = dict()
all_results['XGB'] = dict()

In [37]:
for label in labels:
    data_X = os.path.join(test_X_path, f"test_X_{label}.pkl")
    with open(data_X , "rb") as f:
        test_X = pickle.load(f)
    data_y = os.path.join(test_y_path, f"test_y_{label}.pkl")
    with open(data_y , "rb") as f:
        test_y = pickle.load(f)
    # データセット毎に処理
    for col in columns.keys():
        result = dict()
        # 目的変数毎に処理
        for label in tqdm(labels):
            if len(test_X[columns[col]]) > 0:
                # モデル取得
                pred_model = best_model
                # 学習
                pred_model.fit(train_X[columns[col]].values, train_y)
                # 結果データ作成
                result[label] = test_X[["code"]].copy()
                result[label]["datetime"] = test_X[columns[col]].index
                # 予測
                result[label]["predict"] = pred_model.predict(test_X[columns[col]].values)
                result[label]["predict_dir"] = np.sign(result[label]["predict"])
                # 実際の結果
                result[label]["actual"] = test_y.values
                result[label]["actual_dir"] = np.sign(result[label]["actual"])
                result[label].dropna(inplace=True)

        all_results['XGB'][col] = result

0:	learn: 0.0761455	total: 13ms	remaining: 9.12s
1:	learn: 0.0760797	total: 26.2ms	remaining: 9.13s
2:	learn: 0.0760124	total: 36.4ms	remaining: 8.45s
3:	learn: 0.0759439	total: 44.8ms	remaining: 7.8s
4:	learn: 0.0758769	total: 50.9ms	remaining: 7.07s
5:	learn: 0.0758129	total: 56.4ms	remaining: 6.52s
6:	learn: 0.0757582	total: 62.5ms	remaining: 6.18s
7:	learn: 0.0757004	total: 66.4ms	remaining: 5.74s
8:	learn: 0.0756359	total: 70.9ms	remaining: 5.45s
9:	learn: 0.0755777	total: 74.5ms	remaining: 5.14s
10:	learn: 0.0755208	total: 78.5ms	remaining: 4.92s
11:	learn: 0.0754629	total: 82.8ms	remaining: 4.75s
12:	learn: 0.0754046	total: 87.5ms	remaining: 4.62s
13:	learn: 0.0753481	total: 91.9ms	remaining: 4.5s
14:	learn: 0.0752925	total: 96.4ms	remaining: 4.4s
15:	learn: 0.0752391	total: 101ms	remaining: 4.32s
16:	learn: 0.0751880	total: 105ms	remaining: 4.2s
17:	learn: 0.0751338	total: 108ms	remaining: 4.08s
18:	learn: 0.0750801	total: 111ms	remaining: 3.99s
19:	learn: 0.0750270	total: 115m

161:	learn: 0.0721609	total: 733ms	remaining: 2.44s
162:	learn: 0.0721536	total: 738ms	remaining: 2.43s
163:	learn: 0.0721457	total: 743ms	remaining: 2.43s
164:	learn: 0.0721390	total: 748ms	remaining: 2.42s
165:	learn: 0.0721317	total: 753ms	remaining: 2.42s
166:	learn: 0.0721260	total: 758ms	remaining: 2.42s
167:	learn: 0.0721197	total: 762ms	remaining: 2.41s
168:	learn: 0.0721127	total: 766ms	remaining: 2.41s
169:	learn: 0.0721059	total: 769ms	remaining: 2.4s
170:	learn: 0.0720995	total: 774ms	remaining: 2.39s
171:	learn: 0.0720929	total: 778ms	remaining: 2.39s
172:	learn: 0.0720862	total: 783ms	remaining: 2.38s
173:	learn: 0.0720794	total: 787ms	remaining: 2.38s
174:	learn: 0.0720733	total: 792ms	remaining: 2.38s
175:	learn: 0.0720687	total: 796ms	remaining: 2.37s
176:	learn: 0.0720627	total: 800ms	remaining: 2.36s
177:	learn: 0.0720562	total: 803ms	remaining: 2.35s
178:	learn: 0.0720508	total: 807ms	remaining: 2.35s
179:	learn: 0.0720455	total: 812ms	remaining: 2.35s
180:	learn: 0

334:	learn: 0.0714300	total: 1.58s	remaining: 1.72s
335:	learn: 0.0714274	total: 1.59s	remaining: 1.72s
336:	learn: 0.0714244	total: 1.59s	remaining: 1.72s
337:	learn: 0.0714208	total: 1.6s	remaining: 1.71s
338:	learn: 0.0714180	total: 1.6s	remaining: 1.71s
339:	learn: 0.0714156	total: 1.61s	remaining: 1.7s
340:	learn: 0.0714128	total: 1.61s	remaining: 1.7s
341:	learn: 0.0714103	total: 1.62s	remaining: 1.7s
342:	learn: 0.0714075	total: 1.63s	remaining: 1.69s
343:	learn: 0.0714035	total: 1.63s	remaining: 1.69s
344:	learn: 0.0714006	total: 1.64s	remaining: 1.69s
345:	learn: 0.0713980	total: 1.64s	remaining: 1.68s
346:	learn: 0.0713948	total: 1.65s	remaining: 1.68s
347:	learn: 0.0713917	total: 1.66s	remaining: 1.67s
348:	learn: 0.0713891	total: 1.66s	remaining: 1.67s
349:	learn: 0.0713860	total: 1.66s	remaining: 1.66s
350:	learn: 0.0713837	total: 1.67s	remaining: 1.66s
351:	learn: 0.0713808	total: 1.68s	remaining: 1.66s
352:	learn: 0.0713771	total: 1.68s	remaining: 1.65s
353:	learn: 0.071

503:	learn: 0.0710406	total: 2.43s	remaining: 945ms
504:	learn: 0.0710386	total: 2.44s	remaining: 941ms
505:	learn: 0.0710363	total: 2.44s	remaining: 936ms
506:	learn: 0.0710344	total: 2.45s	remaining: 931ms
507:	learn: 0.0710323	total: 2.45s	remaining: 926ms
508:	learn: 0.0710312	total: 2.46s	remaining: 922ms
509:	learn: 0.0710285	total: 2.46s	remaining: 917ms
510:	learn: 0.0710265	total: 2.47s	remaining: 913ms
511:	learn: 0.0710246	total: 2.47s	remaining: 908ms
512:	learn: 0.0710237	total: 2.48s	remaining: 903ms
513:	learn: 0.0710220	total: 2.48s	remaining: 898ms
514:	learn: 0.0710204	total: 2.49s	remaining: 894ms
515:	learn: 0.0710193	total: 2.49s	remaining: 889ms
516:	learn: 0.0710176	total: 2.5s	remaining: 884ms
517:	learn: 0.0710156	total: 2.5s	remaining: 879ms
518:	learn: 0.0710136	total: 2.51s	remaining: 874ms
519:	learn: 0.0710121	total: 2.51s	remaining: 869ms
520:	learn: 0.0710106	total: 2.51s	remaining: 864ms
521:	learn: 0.0710088	total: 2.52s	remaining: 859ms
522:	learn: 0.

693:	learn: 0.0707226	total: 3.29s	remaining: 28.4ms
694:	learn: 0.0707216	total: 3.29s	remaining: 23.7ms
695:	learn: 0.0707194	total: 3.3s	remaining: 18.9ms
696:	learn: 0.0707171	total: 3.3s	remaining: 14.2ms
697:	learn: 0.0707161	total: 3.31s	remaining: 9.47ms
698:	learn: 0.0707147	total: 3.31s	remaining: 4.74ms
699:	learn: 0.0707138	total: 3.31s	remaining: 0us
0:	learn: 0.0761455	total: 12.3ms	remaining: 8.61s
1:	learn: 0.0760797	total: 23ms	remaining: 8.02s
2:	learn: 0.0760124	total: 34.1ms	remaining: 7.93s
3:	learn: 0.0759439	total: 45ms	remaining: 7.83s
4:	learn: 0.0758769	total: 54.5ms	remaining: 7.58s
5:	learn: 0.0758129	total: 63.1ms	remaining: 7.3s
6:	learn: 0.0757582	total: 70.7ms	remaining: 7s
7:	learn: 0.0757004	total: 77.5ms	remaining: 6.7s
8:	learn: 0.0756359	total: 84.5ms	remaining: 6.49s
9:	learn: 0.0755777	total: 90.8ms	remaining: 6.26s
10:	learn: 0.0755208	total: 97.6ms	remaining: 6.11s
11:	learn: 0.0754629	total: 104ms	remaining: 5.95s
12:	learn: 0.0754046	total: 10

...

673:	learn: 0.0662386	total: 2.75s	remaining: 106ms
674:	learn: 0.0662367	total: 2.76s	remaining: 102ms
675:	learn: 0.0662345	total: 2.76s	remaining: 98.1ms
676:	learn: 0.0662318	total: 2.77s	remaining: 94.1ms
677:	learn: 0.0662298	total: 2.77s	remaining: 90.1ms
678:	learn: 0.0662269	total: 2.78s	remaining: 86ms
679:	learn: 0.0662254	total: 2.78s	remaining: 81.9ms
680:	learn: 0.0662233	total: 2.79s	remaining: 77.8ms
681:	learn: 0.0662210	total: 2.79s	remaining: 73.7ms
682:	learn: 0.0662184	total: 2.8s	remaining: 69.6ms
683:	learn: 0.0662158	total: 2.8s	remaining: 65.6ms
684:	learn: 0.0662141	total: 2.81s	remaining: 61.5ms
685:	learn: 0.0662119	total: 2.81s	remaining: 57.5ms
686:	learn: 0.0662098	total: 2.82s	remaining: 53.3ms
687:	learn: 0.0662060	total: 2.82s	remaining: 49.2ms
688:	learn: 0.0662037	total: 2.83s	remaining: 45.1ms
689:	learn: 0.0661999	total: 2.83s	remaining: 41ms
690:	learn: 0.0661981	total: 2.83s	remaining: 36.9ms
691:	learn: 0.0661962	total: 2.84s	remaining: 32.8ms
6

137:	learn: 0.0692764	total: 577ms	remaining: 2.35s
138:	learn: 0.0692583	total: 582ms	remaining: 2.35s
139:	learn: 0.0692411	total: 587ms	remaining: 2.35s
140:	learn: 0.0692226	total: 593ms	remaining: 2.35s
141:	learn: 0.0692029	total: 598ms	remaining: 2.35s
142:	learn: 0.0691855	total: 603ms	remaining: 2.35s
143:	learn: 0.0691687	total: 607ms	remaining: 2.34s
144:	learn: 0.0691543	total: 612ms	remaining: 2.34s
145:	learn: 0.0691386	total: 615ms	remaining: 2.33s
146:	learn: 0.0691153	total: 619ms	remaining: 2.33s
147:	learn: 0.0690982	total: 623ms	remaining: 2.32s
148:	learn: 0.0690834	total: 627ms	remaining: 2.32s
149:	learn: 0.0690677	total: 634ms	remaining: 2.32s
150:	learn: 0.0690525	total: 639ms	remaining: 2.32s
151:	learn: 0.0690364	total: 646ms	remaining: 2.33s
152:	learn: 0.0690221	total: 650ms	remaining: 2.32s
153:	learn: 0.0690062	total: 655ms	remaining: 2.32s
154:	learn: 0.0689893	total: 659ms	remaining: 2.31s
155:	learn: 0.0689749	total: 663ms	remaining: 2.31s
156:	learn: 

307:	learn: 0.0675997	total: 1.36s	remaining: 1.73s
308:	learn: 0.0675929	total: 1.37s	remaining: 1.73s
309:	learn: 0.0675881	total: 1.37s	remaining: 1.73s
310:	learn: 0.0675827	total: 1.38s	remaining: 1.72s
311:	learn: 0.0675759	total: 1.38s	remaining: 1.72s
312:	learn: 0.0675708	total: 1.39s	remaining: 1.72s
313:	learn: 0.0675658	total: 1.39s	remaining: 1.71s
314:	learn: 0.0675560	total: 1.4s	remaining: 1.71s
315:	learn: 0.0675497	total: 1.4s	remaining: 1.7s
316:	learn: 0.0675438	total: 1.41s	remaining: 1.7s
317:	learn: 0.0675381	total: 1.41s	remaining: 1.7s
318:	learn: 0.0675334	total: 1.42s	remaining: 1.69s
319:	learn: 0.0675265	total: 1.42s	remaining: 1.69s
320:	learn: 0.0675213	total: 1.42s	remaining: 1.68s
321:	learn: 0.0675137	total: 1.43s	remaining: 1.67s
322:	learn: 0.0675061	total: 1.43s	remaining: 1.67s
323:	learn: 0.0675020	total: 1.43s	remaining: 1.67s
324:	learn: 0.0674968	total: 1.44s	remaining: 1.66s
325:	learn: 0.0674917	total: 1.45s	remaining: 1.66s
326:	learn: 0.067

466:	learn: 0.0668782	total: 2.55s	remaining: 1.27s
467:	learn: 0.0668756	total: 2.56s	remaining: 1.27s
468:	learn: 0.0668723	total: 2.58s	remaining: 1.27s
469:	learn: 0.0668688	total: 2.58s	remaining: 1.26s
470:	learn: 0.0668664	total: 2.59s	remaining: 1.26s
471:	learn: 0.0668617	total: 2.6s	remaining: 1.26s
472:	learn: 0.0668592	total: 2.61s	remaining: 1.25s
473:	learn: 0.0668565	total: 2.61s	remaining: 1.25s
474:	learn: 0.0668537	total: 2.62s	remaining: 1.24s
475:	learn: 0.0668515	total: 2.63s	remaining: 1.24s
476:	learn: 0.0668484	total: 2.64s	remaining: 1.24s
477:	learn: 0.0668445	total: 2.66s	remaining: 1.23s
478:	learn: 0.0668385	total: 2.67s	remaining: 1.23s
479:	learn: 0.0668352	total: 2.68s	remaining: 1.23s
480:	learn: 0.0668311	total: 2.69s	remaining: 1.22s
481:	learn: 0.0668279	total: 2.7s	remaining: 1.22s
482:	learn: 0.0668246	total: 2.71s	remaining: 1.22s
483:	learn: 0.0668217	total: 2.71s	remaining: 1.21s
484:	learn: 0.0668185	total: 2.72s	remaining: 1.21s
485:	learn: 0.

627:	learn: 0.0663712	total: 3.53s	remaining: 405ms
628:	learn: 0.0663670	total: 3.54s	remaining: 399ms
629:	learn: 0.0663649	total: 3.54s	remaining: 393ms
630:	learn: 0.0663626	total: 3.54s	remaining: 388ms
631:	learn: 0.0663606	total: 3.55s	remaining: 382ms
632:	learn: 0.0663581	total: 3.56s	remaining: 376ms
633:	learn: 0.0663533	total: 3.56s	remaining: 371ms
634:	learn: 0.0663502	total: 3.56s	remaining: 365ms
635:	learn: 0.0663470	total: 3.57s	remaining: 359ms
636:	learn: 0.0663450	total: 3.57s	remaining: 354ms
637:	learn: 0.0663429	total: 3.58s	remaining: 348ms
638:	learn: 0.0663398	total: 3.58s	remaining: 342ms
639:	learn: 0.0663373	total: 3.59s	remaining: 336ms
640:	learn: 0.0663356	total: 3.59s	remaining: 331ms
641:	learn: 0.0663294	total: 3.6s	remaining: 325ms
642:	learn: 0.0663274	total: 3.6s	remaining: 319ms
643:	learn: 0.0663253	total: 3.61s	remaining: 314ms
644:	learn: 0.0663235	total: 3.61s	remaining: 308ms
645:	learn: 0.0663198	total: 3.62s	remaining: 302ms
646:	learn: 0.

In [38]:
results = []
for model in all_results.keys():
    for col in all_results[model]:
        tmp = pd.concat(all_results[model][col])
        tmp["model"] = model
        tmp["feature"] = col
        results.append(tmp)
results = pd.concat(results)
results["label"] = [x[0] for x in results.index]
results.head(5)

code   datetime   predict  predict_dir   actual  actual_dir model           feature  \
              datetime                                                                                          
label_high_20 2019-02-08  1301 2019-02-08 -0.040598         -1.0  0.00604         1.0   XGB  fundamental_only   
              2019-05-13  1301 2019-05-13 -0.041494         -1.0 -0.06317        -1.0   XGB  fundamental_only   
              2019-08-02  1301 2019-08-02 -0.044052         -1.0 -0.08723        -1.0   XGB  fundamental_only   
              2019-11-05  1301 2019-11-05 -0.043978         -1.0 -0.03800        -1.0   XGB  fundamental_only   
              2020-02-07  1301 2020-02-07 -0.041133         -1.0 -0.17832        -1.0   XGB  fundamental_only   

                                  label  
              datetime                   
label_high_20 2019-02-08  label_high_20  
              2019-05-13  label_high_20  
              2019-08-02  label_high_20  
              2019-11-05  label_high_20  
              2020-02-07  label_high_20

In [39]:
# 結果保存用変数
all_metrics = []

# データセット毎に処理
for feature in columns:
    matrix = dict()
    # 目的変数毎に処理
    for label in labels:
        # 処理対象データに絞り込み
        tmp_df = results[(results["model"] == "XGB") & (results["label"] == label) & (results["feature"] == feature)]
        # RMSE
        rmse = np.sqrt(mean_squared_error(tmp_df["predict"], tmp_df["actual"]))
        # 精度
        accuracy = accuracy_score(tmp_df["predict_dir"], tmp_df["actual_dir"])
        # 相関係数
        corr = np.corrcoef(tmp_df["actual"], tmp_df["predict"])[0, 1]
        # 順位相関
        spearman_corr = spearmanr(tmp_df["actual"], tmp_df["predict"])[0]
        # 結果を保存
        matrix[label] = [rmse, accuracy, spearman_corr,corr, corr**2, feature, model, tmp_df.shape[0]]
    res = pd.DataFrame.from_dict(matrix).T
    res.columns = ["RMSE","accuracy","spearman_corr","corr","R^2 score","feature", "model", "# of samples"]
    all_metrics.append(res)
all_metrics = pd.concat(all_metrics)
all_metrics.reset_index()

,index,RMSE,accuracy,spearman_corr,corr,R^2 score,feature,model,# of samples
0,label_high_20,0.103912,0.867938,0.148663,0.162729,0.0264807,fundamental_only,XGB,32515
1,label_low_20,0.103912,0.867938,0.148663,0.162729,0.0264807,fundamental_only,XGB,32515
2,label_high_20,0.105502,0.867938,0.0894509,0.0742022,0.00550596,return_only,XGB,32515
3,label_low_20,0.105502,0.867938,0.0894509,0.0742022,0.00550596,return_only,XGB,32515
4,label_high_20,0.103835,0.867938,0.168253,0.158312,0.0250626,technical_only,XGB,32515
5,label_low_20,0.103835,0.867938,0.168253,0.158312,0.0250626,technical_only,XGB,32515
6,label_high_20,0.103268,0.867938,0.170305,0.163395,0.0266981,fundamental+technical,XGB,32515
7,label_low_20,0.103268,0.867938,0.170305,0.163395,0.0266981,fundamental+technical,XGB,32515
8,label_high_20,0.103012,0.867938,0.162938,0.154047,0.0237305,selected_columns,XGB,32515
9,label_low_20,0.103012,0.867938,0.162938,0.154047,0.0237305,selected_columns,XGB,32515


In [24]:
tmp = all_metrics.drop(columns=["# of samples"])

In [25]:
tmp.to_csv('result_XGB.csv', index=None)

In [26]:
tmp = pd.read_csv('result_XGB.csv')

In [27]:
tmp.groupby(['feature', 'model']).mean()

,,RMSE,accuracy,spearman_corr,corr,R^2 score
feature,model,,,,,
fundamental+technical,XGB,0.104509,0.864155,0.159079,0.153215,0.023475
fundamental_only,XGB,0.104447,0.866738,0.140670,0.143726,0.020657
return_only,XGB,0.107356,0.867784,0.080475,0.058545,0.003428
technical_only,XGB,0.105467,0.861449,0.153756,0.146159,0.021362
